# 안녕하세요^^ 
# AIVLE 미니 프로젝트3차 팀 미션입니다. 
# 주어진 문제를 잘 읽고 해결하시길 바랍니다.


## 올레TV 예능 콘텐츠 중 하나인 프로그램 "코미디빅리그"의 영상을 분석하고 코너를 분류하는 AI 모델을 만들어 보세요.

"코미디빅리그" 는 tvN에서 방영 중인 서바이벌 형식의 공개 코미디 프로그램으로 3개월 단위로 인기 코너를 선정하는 방식인 쿼제를 도입하고 있다. 
이번 과제는 2022년 1쿼터(1~3월) 방영분 중 2월 방영한 3편을 학습 데이터로 하여 3월 방송의 코너 구간을 추출해내는 것입니다.

아래의 조건들을 참고하여 문제를 해결하세요.
> - 주어진 데이터 ( Train 데이터 :  2월 방영분 3편 / Test 데이터 : 3월 방영분 1편 )
> - 제출 파일은 <font color="red">.ipynb 파일</font>과  조에서 각 과정마다 문제 해결을 위해 진행한 방법을 정리한 <font color="red">발표자료(.pptx)</font>를 포함히여 제출합니다.<br> 
①데이터 분석 및 학습데이터 생성, ②데이터 전처리, ③AI 모델링(아키텍처, 성능비교), ④분류 결과 <br> 
※ 각 과정은 문제 해결 방법에 따라 변경 가능<br><br>
> <b>제출 파일( 압축파일명 : [팀]미니프로젝트3차_AI 0반_00조.zip) </b> <br>
>> ① 코드 : [팀]미니프로젝트3차_AI 0반_00조.ipynb <br>
>> ② 발표자료 : [팀]미니프로젝트3차_AI 0반 00조.pptx <br>




---

# 0. 환경 설정하기

### 1) 구글 드라이브 연결하기

In [ ]:
# 코랩 사용 시 구글 드라이브 연결
# from google.colab import drive
# drive.mount('/content/drive')

### 2) 경로 확인하기

In [1]:
# ROOT_PATH 확인 
import os

if os.getcwd() == '/content' :
  # 구글 드라이브 사용 시 
  ROOT_PATH = "/content/drive/MyDrive/AIVLE_3rd[Team]"
else :
  ROOT_PATH = os.path.abspath('.')

# 비디오 파일이 저장되어 있는 폴더
VIDEO_PATH = ROOT_PATH + "\\video"
IMAGE_PATH = ROOT_PATH + "\\image2"

# 모델 학습을 위한 데이터 
TRAIN_PATH = ROOT_PATH + "\\train"
# 모델 예측을 위한 test 데이터
TEST_PATH = ROOT_PATH + "\\test"

### 3) 영상 데이터 다운로드하기
아래 코드를 실행하세요.

In [ ]:
# test data 다운로드
# !pip install gdown

# import gdown
# import shutil

# google_path = 'https://drive.google.com/uc?id='
# file_id = '1p4up6pPd0NqUIaf6mlc2mq12r6NYGDOI'
# output_name = 'video.zip'

# gdown.download(google_path+file_id,output_name,quiet=False)

# shutil.move("./" + output_name, ROOT_PATH)

# import zipfile

# with zipfile.ZipFile(ROOT_PATH + "/video.zip") as z:
#     z.extractall(ROOT_PATH)


# 미니프로젝트 3차 팀 미션 지금부터 시작하세요!

- 데이터 수집

In [2]:
# 여기에서부터 입력하세요.
import os
import glob
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
video_list = glob.glob(VIDEO_PATH + '\\*.mp4')

print(video_list)

['c:\\Users\\master\\Downloads\\proj5\\Team\\video\\20220220_1.mp4', 'c:\\Users\\master\\Downloads\\proj5\\Team\\video\\20220220_2.mp4', 'c:\\Users\\master\\Downloads\\proj5\\Team\\video\\20220220_3.mp4', 'c:\\Users\\master\\Downloads\\proj5\\Team\\video\\20220220_4.mp4']


In [15]:
for v in video_list:
  video = cv2.VideoCapture(v)

  video_width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
  video_height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
  video_length = video.get(cv2.CAP_PROP_FRAME_COUNT)
  video_fps = video.get(cv2.CAP_PROP_FPS)

  print(os.path.basename(v))
  print('가로: ', video_width)
  print('세로: ', video_height)
  print('총 프레임 수: ', video_length)
  print('FPS: ', video_fps)
  print('영상 길이: %d 초'%round(video_length/video_fps))
  print()

  video.release()

결혼해두목.mp4
가로:  854.0
세로:  480.0
총 프레임 수:  20801.0
FPS:  30.0
영상 길이: 693 초

두분사망토론.mp4
가로:  854.0
세로:  480.0
총 프레임 수:  21990.0
FPS:  30.0
영상 길이: 733 초

사이코러스.mp4
가로:  854.0
세로:  480.0
총 프레임 수:  19015.0
FPS:  30.0
영상 길이: 634 초

코빅엔터.mp4
가로:  854.0
세로:  480.0
총 프레임 수:  25506.0
FPS:  30.0
영상 길이: 850 초



In [4]:
if not os.path.exists(IMAGE_PATH):
    os.mkdir(IMAGE_PATH)

for v in video_list:
  video = cv2.VideoCapture(v)
  TIME_MEASUERMENT_UNIT = 3

  video_name = os.path.basename(v)
  video_name = os.path.splitext(video_name)[0]

  if not os.path.exists(IMAGE_PATH + '\\' + video_name):
    os.mkdir(IMAGE_PATH + '\\' + video_name)

  while video.isOpened():
    ret, frame = video.read()
    
    if ret:
      frame_sec = video.get(cv2.CAP_PROP_POS_MSEC) / 1000
      if frame_sec.is_integer():
        if (frame_sec % TIME_MEASUERMENT_UNIT == 0):
          filename = IMAGE_PATH + '\\' + video_name + '\\' + str(round(frame_sec)) + '.jpg'
          # filename = IMAGE_PATH + '\\' + str(round(frame_sec)) + '.jpg'
          cv2.imwrite(filename, frame) 

    else:
      break

  video.release()